## An overview of quality assurance practices in computational research

Authors: James Davenport, Steven Lamerton, Oliver Laslett, Vincent Knight, James Hetherington

## Introduction

## Basic Testing

- Basic testing
- Floating point
- Testing monte carlo
- Testing parallel code (MPI) (Use MPI4Py to make example)
- Asserting on derived properties (e.g. functional derivatives)


## Further Testing

* How to generate fixtures/examples (property based testing)
* Regression testing with analytic solutions
* Testing workflows
* Visualisation for QE: importance of *seeing* code
* Static analysis and linting (including dimensional and unit assertions)
* Provenance management (recipy stuff)...

## Conclusions